# NOAA Dataset: Handling Missing Values

This notebook details the process for cleaning and preparing the NOAA dataset, with a focus on addressing missing values. The raw dataset contains data from three oceanic buoys, each with different sensor capabilities, leading to two distinct types of missing data: systematic and incidental.

- **Systematic missingness** occurs when a station lacks the hardware to measure a certain variable. In our case, there is a buoy that never reports wind speed which implies it doesn't have an anemometer. 

- **Incidental missingness** refers to small, random gaps in data from a sensor that is otherwise operational, often due to transient sensor malfunctions or transmission errors.

The strategy used here is to first handle the systematic gaps by partitioning the data into analysis-specific subsets. Then, incidental gaps within these subsets are addressed using time-series interpolation. This preserves data integrity by not attempting to impute values that were never physically measurable.

## Setup and Data Loading

In [40]:
import io
import pandas as pd
from IPython.display import display, HTML

In [41]:
files = [
    "../data/source/NOAA_46041.csv",
    "../data/source/NOAA_46050.csv",
    "../data/source/NOAA_46243.csv",
]

dataframes = [pd.read_csv(file) for file in files]
df = pd.concat(dataframes, ignore_index=True)
df["date_time"] = pd.to_datetime(df["date_time"])

## 1. Standardize and Clean Columns

Column names are programmatically standardized for consistency and ease of access. This involves removing leading/trailing whitespace, replacing units and special characters with descriptive suffixes, replacing spaces with underscores, and converting all names to lowercase. A standardized naming convention is best practice to prevent errors, improve code readability, and make data manipulation more predictable.

In [42]:
# Clean up column names
df.columns = df.columns.str.strip()
df.columns = df.columns.str.replace(
    r"\s*\(degrees north\)", "_degrees_north", regex=True
)
df.columns = df.columns.str.replace(r"\s*\(degrees east\)", "_degrees_east", regex=True)
df.columns = df.columns.str.replace(r"\s*\(C°\)", "_celsius", regex=True)
df.columns = df.columns.str.replace(r"\s*\(m/s\)", "_mps", regex=True)
df.columns = df.columns.str.replace(r"\s*\(hPa\)", "_hpa", regex=True)
df.columns = df.columns.str.replace(r"\s*\(s\)", "_s", regex=True)
df.columns = df.columns.str.replace(r"\s*\(m\)", "_m", regex=True)

df.columns = df.columns.str.replace(r"\s+", "_", regex=True)
df.columns = df.columns.str.replace(r"_{2,}", "_", regex=True)
df.columns = df.columns.str.lower()

columns_str = "\n".join(df.columns)

html = f"<pre style='font-family: monospace; border-style: solid; border-width: thin; padding:10px;'>{columns_str}</pre>"
display(HTML(html))

## 2. Merge Redundant Columns

During exploratory data analysis, it was discovered that two columns, `wind_speed_mps` and `wind_speed_cwind_mps`, measure the same physical quantity but are populated for different sets of records. To create a single, authoritative source for wind speed, these columns are merged. The `combine_first` method is used to coalesce the two columns, filling NaN values in the first column with the corresponding values from the second. The original, now-redundant columns are then dropped to simplify the DataFrame.

In [43]:
df["wind_speed"] = df["wind_speed_mps"].combine_first(df["wind_speed_cwind_mps"])

df.drop(columns=["wind_speed_mps", "wind_speed_cwind_mps"], inplace=True)

buffer = io.StringIO()
df.info(buf=buffer)
info = buffer.getvalue()

html = f"<pre style='font-family: monospace; border-style: solid; border-width: thin; padding:10px;'>{info}</pre>"
display(HTML(html))

## 3. Quantify Missing Data

A quantitative analysis of missing data is performed to guide the imputation strategy. The percentage of missing values for each variable is calculated. The results clearly show that variables like `air_temperature_celsius` and `sea_level_pressure_hpa` are missing in over 65% of records. It's a direct result of specific stations lacking the necessary sensors. This confirms the presence of systematic missingness and invalidates the use of simple, global imputation methods, which would introduce significant bias.

In [44]:
missing_data = df.isnull().sum()
missing_percentage = (missing_data / len(df)) * 100
missing_info = pd.DataFrame(
    {"Missing Values": missing_data, "Percentage": missing_percentage}
)

html = f"<pre style='font-family: monospace; border-style: solid; border-width: thin; padding:10px;'>{missing_info}</pre>"
display(HTML(html))

## 4. Data Integration

Initial analyses revealed that the buoys report at different, irregular frequencies. This makes simple partitioning and interpolation problematic, as it can lead to data leakage and an inconsistent time index. To solve this, a better data integration strategy is employed. The goal is to create a single, unified dataframe for the predictive model with a perfectly regular time index. The strategy is as follows:

- **Isolate Stations:** Each station's data is isolated into its own dataframe. Duplicates are handled by grouping by the timestamp.

- **Create a Master Time Index:** A new, perfectly regular 20-minute time index is created. This will serve as the backbone of the final dataframe, ensuring a consistent frequency for our model.

- **Feature Engineering with `merge_asof`:** The data from each buoy is treated as a unique source of features. `pd.merge_asof` is used to get the most recent valid measurement from each buoy for every 20-minute timestamp in the master index. This avoids data leakage and intelligently combines the sparse data.

- **Final Interpolation:** After assembly, `interpolate(method='time')` is used one last time to fill any small, remaining gaps in the final, unified time series.

This approach creates a single, high-quality, feature-rich dataset that uses all available information without corrupting the time series.

In [49]:
# --- 1. Isolate and Clean Each Station's Data Individually ---

# Isolate data for each station
df1 = df[df["station_id"] == 2868187].copy()  # The "master" clock
df2 = df[df["station_id"] == 2868934].copy()  # The fast, 10-min wind station
df3 = df[df["station_id"] == 2888997].copy()  # The wave rider

# Handle potential duplicates by grouping by timestamp and taking the mean.
df1 = df1.groupby("date_time").mean().sort_index()
df2 = df2.groupby("date_time").mean().sort_index()
df3 = df3.groupby("date_time").mean().sort_index()


# --- 2. Build the Model DataFrame from the Master Clock ---

df_complete = df1.copy()


# --- 3. Engineer Features by Merging from Other Buoys ---

# From Buoy 2 (the fast wind station) -> wind speed
df2_features = df2[["wind_speed"]].rename(columns={"wind_speed": "buoy2_wind_speed"})
df_complete = pd.merge_asof(
    left=df_complete,
    right=df2_features,
    left_index=True,
    right_index=True,
    direction="backward",
    tolerance=pd.Timedelta(
        "35 minutes"
    ),  # Find the most recent reading within this window
)

# From Buoy 3 (the wave rider) -> wave height.
df3_features = df3[["significant_wave_height_m"]].rename(
    columns={"significant_wave_height_m": "buoy3_wave_height"}
)
df_complete = pd.merge_asof(
    left=df_complete,
    right=df3_features,
    left_index=True,
    right_index=True,
    direction="backward",
    tolerance=pd.Timedelta("35 minutes"),
)


# --- 4. Handle Missing Values ---

# Now, the only NaNs will be in the NEW columns (`buoy2_wind_speed`, `buoy3_wave_height`)
# if there was no recent reading within the tolerance.
df_complete["buoy2_wind_speed"] = df_complete["buoy2_wind_speed"].interpolate(
    method="time"
)
df_complete["buoy3_wave_height"] = df_complete["buoy3_wave_height"].interpolate(
    method="time"
)

# The primary dataframe's own columns can still be interpolated to handle its
# own incidental missing values. Operating on the full dataframe is not ambiguous.
df_complete.interpolate(method="time", inplace=True)


# --- 5. Final Cleanup ---

# Drop any rows where a value could not be interpolated (usually at the start).
df_complete.dropna(inplace=True)

# Drop the redundant station_id column
if "station_id" in df_complete.columns:
    df_complete.drop(columns="station_id", inplace=True)


# --- 6. Display Final Sampling Rate ---

time_deltas = df_complete.index.to_series().diff().dt.total_seconds() / 60

# Display the frequency of each gap in minutes
gap_counts = time_deltas.value_counts().sort_index()

html = f"""
<p>The analysis of time gaps in <code>df_complete</code> shows the following distribution (in minutes):</p>
<pre style='font-family: monospace; border-style: solid; border-width: thin; padding:10px;'>{gap_counts.to_string()}</pre>
"""
display(HTML(html))

## 5. Save Cleaned Data

In [50]:
buffer = io.StringIO()
df_complete.info(buf=buffer)
info_str = buffer.getvalue()

html = f"""
<p>Final Integrated Model DataFrame Info:</p>
<pre style='font-family: monospace; border-style: solid; border-width: thin; padding:10px;'>{info_str}</pre>
"""
display(HTML(html))

In [47]:
df_complete.to_csv("../data/clean/NOAA_complete.csv")